In [25]:
import requests
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
import re
import pickle
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter
from itertools import chain

In [18]:
from ipynb.fs.full.functions import next_page, clean_date, clean_titulo, sort_df, find_ngrams, bigramas, freq_bigram  

In [3]:
url = 'https://repositorio.unal.edu.co/browse?type=subject&value=32+Ciencia+pol%C3%ADtica+%2F+Political+science'

In [4]:
options = webdriver.ChromeOptions()
#Le agregamos argumentos
options.add_argument('--incognito')
#Esto va a abrir un navegador vacío que dice que un software automatizado controla 
driver = webdriver.Chrome(executable_path='../chromedriver', options=options)

driver.get(url)

In [5]:
def obtener_tesis(driver):
    
    #Creamos una lista vacía para que contenga todas las tesis 
    tesis_total = []
    # Obtenemos los impares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item odd"]'))
    # Obtenemos los pares
    tesis_total.append(driver.find_elements_by_xpath('//li[@class="ds-artifact-item even"]'))
    # Transformamos la lista de listas en una sola lista 
    tesis_total =[t for l in tesis_total for t in l]
    return tesis_total

In [6]:
info = []
def obtener_info(driver):
    tesis_totales = obtener_tesis(driver)
    print('iniciando scraping...')
    for tesis in tesis_totales:
        # Obtenemos fecha
        fecha = tesis.find_element_by_xpath('.//div[@class="artifact-otherdata"]').text
        # Obtenemos título
        titulo = tesis.find_element_by_xpath('.//h4[@class="artifact-title"]').text
        info_dict={'Fecha': fecha, 'Título': titulo}
        info.append(info_dict)
    return info

In [7]:
def main (driver):
    for x in range (1,69):
        obtener_info(driver)
        next_page(driver)
    return info

In [8]:
main(driver)

iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando scraping...
iniciando 

[{'Fecha': 'Universidad Nacional de Colombia Sede Bogotá Dirección Académica Sistema de Patrimonio Cultural y Museos (SPM). 2011',
  'Título': '100 Años de Acción Pública'},
 {'Fecha': 'Universidad Nacional de Colombia Proyectos Temáticos Biblioteca Digital Feminista Ofelia Uribe de Acosta BDF Ciudadanías y democracia. 1997',
  'Título': '40 años del voto de la mujer en Colombia'},
 {'Fecha': 'Universidad Nacional de Colombia Proyectos Temáticos Biblioteca Virtual Colombiana Colección general. 1952',
  'Título': 'A nuestros amigos políticos'},
 {'Fecha': 'Universidad Nacional de Colombia Proyectos Temáticos Biblioteca Virtual Colombiana Colección general. 1982',
  'Título': 'Abel Naranjo Villegas'},
 {'Fecha': 'Universidad Nacional de Colombia Revistas electrónicas UN Revista FORUM. 2015-01-01',
  'Título': 'La academia: de destituyente a constituyente universitaria'},
 {'Fecha': 'Universidad Nacional de Colombia Sede Bogotá Facultad de Derecho, Ciencias Políticas y Sociales. 2017-05-0

In [9]:
driver.close()

Convertimos el archivo info en un DataFrame

In [10]:
df = pd.DataFrame(info)

Cambiamos los valores dentro de fecha a años

In [11]:
regex = r'(\d{1,4})'
clean_date(df, regex)

,Fecha,Título
0,2011,100 Años de Acción Pública
1,1997,40 años del voto de la mujer en Colombia
2,1952,A nuestros amigos políticos
3,1982,Abel Naranjo Villegas
4,2015,La academia: de destituyente a constituyente u...
...,...,...
1342,1981,Yira Castro: mi bandera es la alegría
1343,2012,"Zonas francas, globalización, política pública..."
1344,2015,"Wabgou, Maguemati; Arocha, Jaime; Salgado, Aid..."
1345,2016,What’s love got to do with it? Un análisis crí...


Vemos que acá tenemos NaN para el año de la tesis

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1347 entries, 0 to 1346
Data columns (total 2 columns):
Fecha     1330 non-null object
Título    1347 non-null object
dtypes: object(2)
memory usage: 21.2+ KB


Reemplazamos los NaN con 1, para convertir los otros valores a integrers

In [14]:
df["Fecha"].fillna(1, inplace = True)

Vemos que ya no tenemos NaN

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1347 entries, 0 to 1346
Data columns (total 2 columns):
Fecha     1347 non-null object
Título    1347 non-null object
dtypes: object(2)
memory usage: 21.2+ KB


Volvemos el valor en fecha un integrer

In [16]:
df['Fecha'] = df['Fecha'].astype(int)

Limpiamos los títulos de las tesis de signos de puntuación y signos speciales. Además se tokenizan los títulos y se ponen todas las palabras en minúscula

In [42]:
def clean_titulo(df):
    """Esta función recibe un Data Frame y a la columna con """
    # Definimos los signos de puntiación en una variable
    punt_espa = string.punctuation + "–¡¿”“•\r"
    
    # Definimos función para quitar los signos de puntiación 
    def sin_punt(x):
        x = ''.join([i for i in x if i not in (punt_espa)])
        return x
    
    # Aplicamos la función para quitar los signos
    df['Título cleaned'] = df['Título'].apply(sin_punt)
    
    # Definimos las stopwords
    stopwords_esp = stopwords.words('spanish')
    stopwords_esp = stopwords_esp + ['estudio', 'caso', 'análisis', 'bogotá', 'periodo', 'incidencia', 'colombia', 'político', 'hacia', 'impacto', 'the', 'and', 'aproximación', 'influencia', 'factores', 'medio', 'interés', 'programa', 'influencia', 'partir', 'colombiano', 'relación', 'marco', 'concepto', 'gobierno', 'periodo', 'departamento', 'ciudad','What’s', 'love', 'got', 'to', 'do', 'with', 'it', 'of', 'in', 'enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre', 'ddr']
    
    # Ponemos las palabras todas en minúscula y las separamos
    df['Título cleaned'] = df['Título cleaned'].str.lower().str.split()
    
    # Eliminamos las stopwords
    df['Título cleaned'] = df['Título cleaned'].apply(lambda x: [item for item in x if item not in stopwords_esp])
    
    
    return df

In [43]:
clean_titulo(df)

,Fecha,Título,Título cleaned,bigrams
0,2011,100 Años de Acción Pública,"[100, años, acción, pública]","[(100, años), (años, acción), (acción, pública)]"
1,1997,40 años del voto de la mujer en Colombia,"[40, años, voto, mujer]","[(40, años), (años, voto), (voto, mujer)]"
2,1952,A nuestros amigos políticos,"[amigos, políticos]","[(amigos, políticos)]"
3,1982,Abel Naranjo Villegas,"[abel, naranjo, villegas]","[(abel, naranjo), (naranjo, villegas)]"
4,2015,La academia: de destituyente a constituyente u...,"[academia, destituyente, constituyente, univer...","[(academia, destituyente), (destituyente, cons..."
...,...,...,...,...
1342,1981,Yira Castro: mi bandera es la alegría,"[yira, castro, bandera, alegría]","[(yira, castro), (castro, bandera), (bandera, ..."
1343,2012,"Zonas francas, globalización, política pública...","[zonas, francas, globalización, política, públ...","[(zonas, francas), (francas, globalización), (..."
1344,2015,"Wabgou, Maguemati; Arocha, Jaime; Salgado, Aid...","[wabgou, maguemati, arocha, jaime, salgado, ai...","[(wabgou, maguemati), (maguemati, arocha), (ar..."
1345,2016,What’s love got to do with it? Un análisis crí...,"[what’s, crítico, feminista, amor, parejas, he...","[(what’s, crítico), (crítico, feminista), (fem..."


Extraemos los bigramas de los títulos

In [44]:
bigramas(df)

,Fecha,Título,Título cleaned,bigrams
0,2011,100 Años de Acción Pública,"[100, años, acción, pública]","[(100, años), (años, acción), (acción, pública)]"
1,1997,40 años del voto de la mujer en Colombia,"[40, años, voto, mujer]","[(40, años), (años, voto), (voto, mujer)]"
2,1952,A nuestros amigos políticos,"[amigos, políticos]","[(amigos, políticos)]"
3,1982,Abel Naranjo Villegas,"[abel, naranjo, villegas]","[(abel, naranjo), (naranjo, villegas)]"
4,2015,La academia: de destituyente a constituyente u...,"[academia, destituyente, constituyente, univer...","[(academia, destituyente), (destituyente, cons..."
...,...,...,...,...
1342,1981,Yira Castro: mi bandera es la alegría,"[yira, castro, bandera, alegría]","[(yira, castro), (castro, bandera), (bandera, ..."
1343,2012,"Zonas francas, globalización, política pública...","[zonas, francas, globalización, política, públ...","[(zonas, francas), (francas, globalización), (..."
1344,2015,"Wabgou, Maguemati; Arocha, Jaime; Salgado, Aid...","[wabgou, maguemati, arocha, jaime, salgado, ai...","[(wabgou, maguemati), (maguemati, arocha), (ar..."
1345,2016,What’s love got to do with it? Un análisis crí...,"[what’s, crítico, feminista, amor, parejas, he...","[(what’s, crítico), (crítico, feminista), (fem..."


Guardamos el data frame en un archivo pickle para después trabajar con él

In [45]:
df.to_pickle('repo_nacho.pkl')